```
Copyright 2021 IBM Corporation

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

     http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
```

# Support Vector Machine on Allstate Dataset

## Background 

The goal of this competition is to predict Bodily Injury Liability Insurance claim payments based on the characteristics of the insured’s vehicle. 

## Source

The raw dataset can be obtained directly from the [Allstate Claim Prediction Challenge](https://www.kaggle.com/c/ClaimPredictionChallenge).

In this example, we download the dataset directly from Kaggle using their API. In order for to work work, you must:
1. Login into Kaggle and accept the [competition rules](https://www.kaggle.com/c/ClaimPredictionChallenge/rules).
2. Folow [these instructions](https://www.kaggle.com/docs/api) to install your API token on your machine.

## Goal
The goal of this notebook is to illustrate how Snap ML can accelerate training of a support vector machine model on this dataset.

## Code

In [1]:
cd ../../

/localhome/tpa/snapml-examples/examples


In [2]:
CACHE_DIR='cache-dir'

In [3]:
import numpy as np
import time
from datasets import Allstate
from sklearn.svm import LinearSVC
from snapml import SupportVectorMachine as SnapSupportVectorMachine
from sklearn.metrics import accuracy_score as score

In [4]:
dataset = Allstate(cache_dir=CACHE_DIR)
X_train, X_test, y_train, y_test = dataset.get_train_test_split()

Reading binary Allstate dataset (cache) from disk.


In [5]:
print("Number of examples: %d" % (X_train.shape[0]))
print("Number of features: %d" % (X_train.shape[1]))
print("Number of classes:  %d" % (len(np.unique(y_train))))

Number of examples: 9229003
Number of features: 87
Number of classes:  2


In [6]:
# the dataset is highly imbalanced
labels, sizes = np.unique(y_train, return_counts=True)
print("%6.2f %% of the training transactions belong to class 0" % (sizes[0]*100.0/(sizes[0]+sizes[1])))
print("%6.2f %% of the training transactions belong to class 1" % (sizes[1]*100.0/(sizes[0]+sizes[1])))

 99.27 % of the training transactions belong to class 0
  0.73 % of the training transactions belong to class 1


In [7]:
model = LinearSVC(loss="hinge", class_weight='balanced', fit_intercept=False, random_state=42)
t0 = time.time()
model.fit(X_train, y_train)
t_fit_sklearn = time.time()-t0
score_sklearn = score(y_test, model.predict(X_test))
print("Training time (sklearn): %6.2f seconds" % (t_fit_sklearn))
print("Accuracy score (sklearn): %.4f" % (score_sklearn))

/localhome/tpa/anaconda3/envs/snapenv/lib/python3.8/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Training time (sklearn): 111.37 seconds
Accuracy score (sklearn): 0.7851


In [8]:
model = SnapSupportVectorMachine(n_jobs=4, class_weight='balanced', fit_intercept=False, random_state=42)
t0 = time.time()
model.fit(X_train, y_train)
t_fit_snapml = time.time()-t0
score_snapml = score(y_test, model.predict(X_test))
print("Training time (snapml): %6.2f seconds" % (t_fit_snapml))
print("Accuracy score (snapml): %.4f" % (score_snapml))

Training time (snapml):  14.98 seconds
Accuracy score (snapml): 0.7860


In [9]:
speed_up = t_fit_sklearn/t_fit_snapml
score_diff = (score_snapml-score_sklearn)/score_sklearn
print("Speed-up:                %.1f x" % (speed_up))
print("Relative diff. in score: %.4f" % (score_diff))

Speed-up:                7.4 x
Relative diff. in score: 0.0012


## Disclaimer

Performance results always depend on the hardware and software environment. 

Information regarding the environment that was used to run this notebook are provided below:

In [10]:
import utils
environment = utils.get_environment()
for k,v in environment.items():
    print("%15s: %s" % (k, v))

       platform: Linux-4.15.0-151-generic-x86_64-with-glibc2.10
      cpu_count: 40
   cpu_freq_min: 800.0
   cpu_freq_max: 2101.0
   total_memory: 250.58893203735352
 snapml_version: 1.8.3
sklearn_version: 1.0.1
xgboost_version: 1.3.3
lightgbm_version: 3.1.1


## Record Statistics

Finally, we record the enviroment and performance statistics for analysis outside of this standalone notebook.

In [11]:
import scrapbook as sb
sb.glue("result", {
    'dataset': dataset.name,
    'n_examples_train': X_train.shape[0],
    'n_examples_test': X_test.shape[0],
    'n_features': X_train.shape[1],
    'n_classes': len(np.unique(y_train)),
    'model': type(model).__name__,
    'score': score.__name__,
    't_fit_sklearn': t_fit_sklearn,
    'score_sklearn': score_sklearn,
    't_fit_snapml': t_fit_snapml,
    'score_snapml': score_snapml,
    'score_diff': score_diff,
    'speed_up': speed_up,
    **environment,
})

/localhome/tpa/anaconda3/envs/snapenv/lib/python3.8/site-packages/papermill/iorw.py:50: FutureWarning: pyarrow.HadoopFileSystem is deprecated as of 2.0.0, please use pyarrow.fs.HadoopFileSystem instead.
  from pyarrow import HadoopFileSystem
